In [142]:
from astropy.io.votable import parse
import pandas as pd
import numpy as np

In [15]:
def votable_to_pandas(votable_file):
    votable = parse(votable_file)
    table = votable.get_first_table().to_table(use_names_over_ids=True)
    return table.to_pandas()

In [124]:
file = 'votable_with_Teff.txt'
lines = open(file,'r').read().split('\nGAIA')
print(len(lines))

561


In [130]:
all_data = []
for i,line in enumerate(lines):
    if 'DR2' in line and 'Fe_H' in line:
        data, feh_list = line.split('|Fe_H')
        data = data.split('|')
        data[0] = data[0].split(' ')[2]
        if len(data[1])>0 and len(data[3])>0:
            data_dict={}
            data_dict['ID'] = data[0]
            data_dict['B_mag'] = float(data[1])
            try: data_dict['B_error'] = float(data[2])
            except ValueError: data_dict['B_error'] = None
            data_dict['V_mag'] = float(data[3])
            try: data_dict['V_error'] = float(data[4])
            except ValueError: data_dict['V_error'] = None
            data_dict['plx'] = float(data[5])
            data_dict['plx_error'] = float(data[6])
            
            if '\n' in feh_list:
                feh_list=feh_list.split('\n')
            else:
                feh_list = [feh_list]
            feh_array = []
            for feh in feh_list:
                feh = feh.split('|')[1:-1]
                feh = [fehi.strip() for fehi in feh]
                if len(feh[0])>0:
                    if ' ' in feh[0]:
                        Teff = float(feh[0].split(' ')[0])
                        if Teff<1000:
                            continue
                    else:
                        if float(feh[0])>1000:
                            Teff = float(feh[0])
                        else: continue
                else: continue
                if len(feh[1])>0:
                    try: FeH = float(feh[1])
                    except ValueError: FeH = None
                else:
                    FeH = None
                ref = feh[-1]
                feh_array.append([Teff, FeH, ref])
            if len(feh_array)==0:
                continue
            data_dict['feh_table'] = feh_array
            all_data.append(data_dict)

print(len(all_data))

119


In [131]:
df = pd.DataFrame(all_data)

In [133]:
length=[]
for i in list(df['feh_table']):
    length.append(len(i))
df['feh_count'] = length
df = df[['ID','B_mag','B_error','V_mag','V_error','plx','plx_error','feh_table','feh_count']]
df

,ID,B_mag,B_error,V_mag,V_error,plx,plx_error,feh_table,feh_count
0,605001329677769088,15.080,NaN,14.510,0.121,1.1437,0.0295,"[[5836.0, 0.05, 2011AJ....141...89S]]",1
1,604905500367349888,13.734,0.096,12.951,0.056,1.1357,0.0429,"[[5458.0, None, 2013AJ....146..134K]]",1
2,604895089366706304,14.770,NaN,14.069,0.095,1.1442,0.0300,"[[5718.0, -0.06, 2011AJ....141...89S]]",1
3,604943880195096320,14.990,NaN,14.461,0.086,1.0975,0.0352,"[[5904.0, 0.06, 2011AJ....141...89S], [5957.0,...",2
4,604919480485158784,12.800,0.360,11.230,0.110,1.2565,0.0365,"[[4763.0, -0.01, 2016A&A...594A..43H], [4779.0...",4
5,604922061761107584,15.810,NaN,15.094,0.030,1.1543,0.0380,"[[5551.0, -0.06, 2011AJ....141...89S], [5642.0...",2
6,604911096708974080,15.460,NaN,14.762,0.180,1.1991,0.0345,"[[4750.0, None, 2013AJ....146..134K]]",1
7,604924531366686080,14.630,NaN,14.137,0.056,1.0982,0.0312,"[[6067.0, -0.03, 2011AJ....141...89S], [6114.0...",2
8,604917217038188288,12.530,NaN,11.394,0.037,1.1694,0.0490,"[[4800.0, -0.09, 1988PASP..100.1489G], [4790.0...",8
9,604907390153062272,11.760,0.120,11.178,0.067,1.1723,0.0436,"[[4787.0, 0.119, 2016ApJ...823..114N]]",1


In [135]:
df.to_csv('SIMBAD_output.csv',index=False)

In [155]:
ref_list=[]
for i in all_data:
    refs=[]
    for j in i['feh_table']:
        refs.append(j[2])
    refs=list(set(refs))
    ref_list+=refs
print(len(ref_list))
ref_set = set(ref_list)
ref_count = np.zeros(len(ref_set))
for i in ref_list:
    for j,r in enumerate(ref_set):
        if i==r:
            ref_count[j]+=1
ref_df = pd.DataFrame({'ref':list(ref_set),'count':ref_count})
ref_df.sort_values('count',inplace=True,ascending=False)
ref_df

272


,ref,count
5,2011AJ....141...89S,52.0
11,2008AJ....136.2050L,39.0
12,2013AJ....146..134K,28.0
10,2016ApJ...823..114N,19.0
0,2016A&A...589A.115S,15.0
1,2016A&A...594A..43H,15.0
7,2013AJ....146..133M,15.0
26,2011AJ....142...59J,13.0
22,2014A&A...562A.102O,11.0
16,2006A&A...450..557R,10.0
